## Wordy Motivation

A Byte Pair Encoding (BPE) tokenizer splits text into subword units based on the most frequent character pairs in a corpus,\
allowing it to balance vocabulary size and represent rare words efficiently. It starts with individual bytes or characters \
and repeatedly merges the most common adjacent pairs into new tokens until a fixed vocabulary size is reached.

Modern BPE tokenizers used for training and inference in large language models typically apply regex-based pretokenization. \
This step splits text into linguistically or visually meaningful chunks (like words or punctuation groups), preventing merges \
across token boundaries that could produce spurious or misleading tokens (e.g., treating "dog" and "dog!" as entirely different tokens).

Regex pretokenization also enables more efficient frequency counting: if a word like “text” appears 10 times, we can increment pair counts (like 't','e') by 10 directly.\
When a merge occurs (e.g., 't','e','x','t' → 'te','x','t'), only the keys in the frequency dictionary need to be updated — the total count remains the same, \
which simplifies and speeds up the BPE merge step.

In [2]:
import regex as re

In [ ]:
# from here: https://github.com/openai/tiktoken/pull/234/files
# Using this pattern re.finditer will produce one pretoken per group
GPT2_SPLIT_PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
pattern = re.compile(GPT2_SPLIT_PAT)

## Pieces of Training

In [18]:
# mix of languages + emoji
text = "Привет, world! 😄 Let's go, 今日は."

In [29]:
# a bit larger piece of text
text = """Alice looked at the glowing sign: “Добро пожаловать!” — it blinked beneath a line of Chinese characters: 欢迎光临.

She typed quickly: `hello_世界123! :)` — mixing English, symbols, digits, and emojis into her message.  
The response came instantly: "Принято. ✅"  
She smiled, whispered «行吧», and pressed Send.
"""

In [42]:
# and another one from cs336 assignment
text = """ low low low low low lower lower widest widest widest newest newest newest newest newest newest"""

In [ ]:
pattern.findall(text)

[' low',
 ' low',
 ' low',
 ' low',
 ' low',
 ' lower',
 ' lower',
 ' widest',
 ' widest',
 ' widest',
 ' newest',
 ' newest',
 ' newest',
 ' newest',
 ' newest',
 ' newest']

In [187]:
# as described above this dictionary will keep pretoken counts
# tuple(bytes('Привет', encoding='utf-8')) -> integer represented byte sequence for convenience
pretokens = dict()
for mt in pattern.finditer(text):
  pt = mt.group() # -> str; match will have one pretoken per group
  pt = tuple(pt.encode('utf-8'))
  pretokens[pt] = pretokens.get(pt, 0) + 1

next_ix = 256

In [188]:
merges = [int.to_bytes(i) for i in range(256)]

In [252]:
for pt, cnt in pretokens.items():
  print([merges[i] for i in pt], ':', cnt)

[b' ', b'w', b'i', b'd', b'est'] : 3
[b' low'] : 5
[b' low', b'e', b'r'] : 2
[b' ', b'ne', b'west'] : 6


In [253]:
pretokens

{(32, 119, 105, 100, 257): 3, (260,): 5, (260, 101, 114): 2, (32, 262, 261): 6}

In [254]:
# we need to iterate through pretokens to find pair frequencies
pair_counts = dict()
for pt, cnt in pretokens.items():
  for p in zip(pt, pt[1:]):
    pair_counts[p] = pair_counts.get(p, 0) + cnt

In [255]:
for p, cnt in pair_counts.items():
  print(merges[p[0]], '+' ,merges[p[1]], ":", cnt)

b' ' + b'w' : 3
b'w' + b'i' : 3
b'i' + b'd' : 3
b'd' + b'est' : 3
b' low' + b'e' : 2
b'e' + b'r' : 2
b' ' + b'ne' : 6
b'ne' + b'west' : 6


In [256]:
# find most frequent pair, ties resolved in lexicographical order
top_pair, top_cnt = max(pair_counts.items(), key=lambda it: [it[1], it[0]])

In [257]:
top_pair

(262, 261)

In [258]:
print(merges[top_pair[0]], ',', merges[top_pair[1]], '->', top_cnt)

b'ne' , b'west' -> 6


In [259]:
# merge `pair` to become `new_ix` if it's in the `seq`
def merge(seq, pair, new_ix):
  new_seq = []
  i = 0
  while i < len(seq):
    # check in range and if match
    if i+1 < len(seq) and (seq[i], seq[i+1]) == pair:
      new_seq.append(new_ix)
      i += 2 # correct step
    else:
      new_seq.append(seq[i]) # only current position
      i += 1
  return tuple(new_seq)

In [260]:
# Each merge introduces a new token (pair → new token) that wasn’t in the vocabulary before
# Pretoken keys are sequences of current tokens.
# Until you merge ('t', 'e') into 'te', there's no way 'te' appears as a unit inside any key
# Only keys that contain the exact pair ('t', 'e') in adjacent positions will be modified.
# The output of merge() depends deterministically on the input key.
# Therefore, at most one original key can produce any given new_pt in the merge step.
for pt in list(pretokens): # static copy of keys (prevents RuntimeError if we iterate original dict)
  new_pt = merge(pt, top_pair, next_ix)
  if new_pt != pt: # update only if we merged new index
    # even though we proved it can't happen (see above), we want this assertions and perhaps test against it
    # so we are sure not to mess up with implementation
    assert new_pt not in pretokens, f"Collision: {new_pt} already in pretokens"
    pretokens[new_pt] = pretokens.pop(pt) #  safe from key collisions under the BPE merge assumptions (see above)
  
# update merges
merges.append(merges[top_pair[0]] + merges[top_pair[1]])
next_ix += 1

In [262]:
# we can take a look into newly formed tokens
for i, bp in enumerate(merges[256:], 256):
  print(i, '->', bp)

256 -> b'st'
257 -> b'est'
258 -> b'ow'
259 -> b'low'
260 -> b' low'
261 -> b'west'
262 -> b'ne'
263 -> b'newest'


## Let's Put It All Together

In [ ]:
# merge `pair` to become `new_ix` if it's in the `seq`
def merge(seq, pair, new_ix):
  new_seq = []
  i = 0
  while i < len(seq):
    # check in range and if match
    if i+1 < len(seq) and (seq[i], seq[i+1]) == pair:
      new_seq.append(new_ix)
      i += 2 # correct step
    else:
      new_seq.append(seq[i]) # only current position
      i += 1
  return tuple(new_seq)

In [263]:
merges = [int.to_bytes(i) for i in range(256)]

# as described above this dictionary will keep pretoken counts
# tuple(bytes('Привет', encoding='utf-8')) -> integer represented byte sequence for convenience
pretokens = dict()
for mt in pattern.finditer(text):
  pt = mt.group() # -> str; match will have one pretoken per group
  pt = tuple(pt.encode('utf-8'))
  pretokens[pt] = pretokens.get(pt, 0) + 1

next_ix = 256
num_merges = 10

In [264]:
sep = "==================================="
for _ in range(num_merges):
  print(sep)
  # show pretokens
  for pt, cnt in pretokens.items():
    print([merges[i] for i in pt], ':', cnt)
  
  # we need to iterate through pretokens to find pair frequencies
  pair_counts = dict()
  for pt, cnt in pretokens.items():
    for p in zip(pt, pt[1:]):
      pair_counts[p] = pair_counts.get(p, 0) + cnt
  # find most frequent pair, ties resolved in lexicographical order
  top_pair, top_cnt = max(pair_counts.items(), key=lambda it: [it[1], it[0]])
  print("top pair", merges[top_pair[0]], ',', merges[top_pair[1]], '->', top_cnt)
  
  # merge
  for pt in list(pretokens): # static copy of keys (prevents RuntimeError if we iterate original dict)
    new_pt = merge(pt, top_pair, next_ix)
    if new_pt != pt: # update only if we merged new index
      # even though we proved it can't happen (see above), we want this assertions and perhaps test against it
      # so we are sure not to mess up with implementation
      assert new_pt not in pretokens, f"Collision: {new_pt} already in pretokens"
      pretokens[new_pt] = pretokens.pop(pt) #  safe from key collisions under the BPE merge assumptions (see above)
  
  # update merges
  merges.append(merges[top_pair[0]] + merges[top_pair[1]])
  next_ix += 1

print(sep)
# we can take a look into newly formed tokens
for i, bp in enumerate(merges[256:], 256):
  print(i, '->', bp)

[b' ', b'l', b'o', b'w'] : 5
[b' ', b'l', b'o', b'w', b'e', b'r'] : 2
[b' ', b'w', b'i', b'd', b'e', b's', b't'] : 3
[b' ', b'n', b'e', b'w', b'e', b's', b't'] : 6
top pair b's' , b't' -> 9
[b' ', b'l', b'o', b'w'] : 5
[b' ', b'l', b'o', b'w', b'e', b'r'] : 2
[b' ', b'w', b'i', b'd', b'e', b'st'] : 3
[b' ', b'n', b'e', b'w', b'e', b'st'] : 6
top pair b'e' , b'st' -> 9
[b' ', b'l', b'o', b'w'] : 5
[b' ', b'l', b'o', b'w', b'e', b'r'] : 2
[b' ', b'w', b'i', b'd', b'est'] : 3
[b' ', b'n', b'e', b'w', b'est'] : 6
top pair b'o' , b'w' -> 7
[b' ', b'w', b'i', b'd', b'est'] : 3
[b' ', b'n', b'e', b'w', b'est'] : 6
[b' ', b'l', b'ow'] : 5
[b' ', b'l', b'ow', b'e', b'r'] : 2
top pair b'l' , b'ow' -> 7
[b' ', b'w', b'i', b'd', b'est'] : 3
[b' ', b'n', b'e', b'w', b'est'] : 6
[b' ', b'low'] : 5
[b' ', b'low', b'e', b'r'] : 2
top pair b' ' , b'low' -> 7
[b' ', b'w', b'i', b'd', b'est'] : 3
[b' ', b'n', b'e', b'w', b'est'] : 6
[b' low'] : 5
[b' low', b'e', b'r'] : 2
top pair b'w' , b'est' -> 6
[b' 

In [ ]:
special_tokens = ["<|endoftext|>", "<|endofline|>"]
delim = "|".join(map(re.escape, special_tokens))

<\|endoftext\|>|<\|endofline\|>


In [25]:
with open("data/TinyStoriesV2-GPT4-valid.txt", mode='r', encoding="utf-8") as f:
  ts_text = f.read()

In [27]:
len(ts_text)

22493387

In [34]:
ts_text[:400]

'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n<|endoftext|>\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red b'

In [28]:
stories = re.split(delim, ts_text)

In [29]:
len(stories)

27631

In [32]:
stories[2343]

'\nOnce upon a time, there was a little boy named Tim. Yesterday, he went to a shop with his mom. Tim saw a big, red ball. He wanted it so much. He asked his mom, "Can I have the ball, please?" His mom said, "Yes, you can have it."\nTim was so happy. He played with the ball all day. But then, he kicked the ball too hard. The ball flew away and hit a tree. The tree was hurt. The tree said, "Ouch! That hurt! Please be careful next time."\nTim felt sorry for the tree. He said, "I\'m sorry, tree. I will be more careful." He picked up the ball and took it home. Tim\'s mom ordered a new, soft ball for the tree. The tree was happy, and Tim learned to play more carefully. They all lived happily ever after.\n'

In [ ]:
"".join(stories[:3])

'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\nTom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\nSam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\nThey went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could hear them. 

In [ ]:
"".join(stories[:3])

'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\nTom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\nSam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\nThey went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could hear them. 

In [39]:
bytes('<|endoftext|>', encoding="utf-8")

b'<|endoftext|>'

In [40]:
t = "Hello<tok>world<tok><eod>!"
special_tokens = ["<tok>", "<eod>"]
delim = "|".join(map(re.escape, special_tokens))
re.sub(f"(?:{delim})+", " ", t)

'Hello world !'

## Optimization

1. We can do merge and count updates on only sequences really containing top pair -> need pair_to_sequence mapping Dict[tuple[int, int], Set[tuple[int, ...], ...]]

2. We can use heap to keep track of top pairs for O(log n) max pair look up. Real life training can lead to a really large number of unique pairs. Number of possible unique pairs depends on vocab size, which grows every merge iteration (256 -> vocab_size). If vocab_size = n, we have P(n, 2) = n! / (n - 2)! 

Important note: we shouldn't forget about updating counts for overlapping pairs before and after merge, as well as merged top pair itself


In [1]:
with open("data/TinyStoriesV2-GPT4-valid.txt", mode='r', encoding="utf-8") as f:
  ts_text = f.read()

In [4]:
text = ts_text[:200]

In [11]:
# we did step-by-step training on this text, so for the first test we should see that optimized
# version reproduces naive implementation results.
text = """ low low low low low lower lower widest widest widest newest newest newest newest newest newest"""

In [12]:
type(text), len(text)

(str, 95)

In [9]:
# same pre-token counter
from toksmith.tokenizer import Tokenizer, _merge

In [10]:
tok = Tokenizer()

In [13]:
# Data structures
# I am using byte-unit term to underline that tokens are formed as a sequences of units, which correspond to either
# one byte or multiple bytes
# 1. pretoken_count: Dict[tuple[int, ...], int] = pretokens (as byte-unit sequences) frequency dictionary
# 2. pairs_count: Dict[tuple[int, int], int] = count of all sequential pairs of byte-units in the train corpus 
# 3. pair_to_pretoken: Dict[tuple[int, int], Set[tuple[int, ...], ...]] = map a pair to all pretokens, which have it
# this data structure will allow us to only touch relevant pretokens during merge iteration (and not all of them as
# in our naive implementation).
# 4. max heap on List[tuple[int, tuple[int, int]], ...] = provides a O(log n) access to most frequent pair

In [22]:
from collections import defaultdict
import heapq

In [63]:
def merge(seq, pair, new_ix, in_pairs):
  if not isinstance(pair, tuple) or len(pair) != 2:
    raise ValueError('`pair` must be a 2-tuple')
  new_seq = []
  out_pairs = set()
  i = 0
  while i < len(seq):
    # check in range and if match
    if i + 1 < len(seq) and (seq[i], seq[i + 1]) == pair:
      new_seq.append(new_ix)
      # if our pair is (x,y), we want to keep track of all (u,x) and (y,v) pairs
      # as well as incoming (u, new_ix) and (new_ix, v)
      # add outgoing and incoming pairs if any
      if i - 1 >= 0:
        # have (u,x) and (u, new_ix)
        out_pairs.add((seq[i-1], seq[i]))
        in_pairs.add((seq[i-1], new_ix))
      if i + 2 < len(seq):
        # have (y,v) and (new_ix, v)
        out_pairs.add((seq[i+1], seq[i+2]))
        in_pairs.add((new_ix, seq[i+2]))
      i += 2  # correct step
    else:
      new_seq.append(seq[i])  # only current position
      i += 1
  return tuple(new_seq), out_pairs, in_pairs

In [72]:
# vocab if we need to print
vocab = {i : bytes([i]) for i in range(256)}
merges = []

In [73]:
# Setup pretoken count TODO: make pretoken_count defaultdict
pretoken_count = tok._pretoken_count(text)

In [74]:
pretoken_count

{(32, 108, 111, 119): 5,
 (32, 108, 111, 119, 101, 114): 2,
 (32, 119, 105, 100, 101, 115, 116): 3,
 (32, 110, 101, 119, 101, 115, 116): 6}

In [75]:
# Setup pairs_count and pair_to_pretoken
pairs_count = defaultdict(int)
pair_to_pretoken = defaultdict(set)
# basically the same code as in our _pairs_count
for pt, cnt in pretoken_count.items():
  for pair in zip(pt, pt[1:]):
    pairs_count[pair] += cnt        # pair frequency
    pair_to_pretoken[pair].add(pt)  # pretoken the pair belongs to


In [76]:
pairs_count

defaultdict(int,
            {(32, 108): 7,
             (108, 111): 7,
             (111, 119): 7,
             (119, 101): 8,
             (101, 114): 2,
             (32, 119): 3,
             (119, 105): 3,
             (105, 100): 3,
             (100, 101): 3,
             (101, 115): 9,
             (115, 116): 9,
             (32, 110): 6,
             (110, 101): 6,
             (101, 119): 6})

In [77]:
pair_to_pretoken

defaultdict(set,
            {(32, 108): {(32, 108, 111, 119), (32, 108, 111, 119, 101, 114)},
             (108, 111): {(32, 108, 111, 119), (32, 108, 111, 119, 101, 114)},
             (111, 119): {(32, 108, 111, 119), (32, 108, 111, 119, 101, 114)},
             (119, 101): {(32, 108, 111, 119, 101, 114),
              (32, 110, 101, 119, 101, 115, 116)},
             (101, 114): {(32, 108, 111, 119, 101, 114)},
             (32, 119): {(32, 119, 105, 100, 101, 115, 116)},
             (119, 105): {(32, 119, 105, 100, 101, 115, 116)},
             (105, 100): {(32, 119, 105, 100, 101, 115, 116)},
             (100, 101): {(32, 119, 105, 100, 101, 115, 116)},
             (101, 115): {(32, 110, 101, 119, 101, 115, 116),
              (32, 119, 105, 100, 101, 115, 116)},
             (115, 116): {(32, 110, 101, 119, 101, 115, 116),
              (32, 119, 105, 100, 101, 115, 116)},
             (32, 110): {(32, 110, 101, 119, 101, 115, 116)},
             (110, 101): {(32, 110, 101, 1

In [78]:
# Setup a heap of (-cnt, -pair), negation is to maintain it as max heap (python defaults to min heap)
# -pair is to maintain the same tie resolution as in our naive implementation
heap = [(-cnt, (-p[0], -p[1])) for p, cnt in pairs_count.items()]
heapq.heapify(heap)

In [79]:
heap

[(-9, (-115, -116)),
 (-9, (-101, -115)),
 (-7, (-111, -119)),
 (-8, (-119, -101)),
 (-7, (-108, -111)),
 (-6, (-110, -101)),
 (-6, (-101, -119)),
 (-3, (-105, -100)),
 (-3, (-100, -101)),
 (-7, (-32, -108)),
 (-2, (-101, -114)),
 (-6, (-32, -110)),
 (-3, (-32, -119)),
 (-3, (-119, -105))]

In [ ]:
# Setup index
new_ix = 256

In [101]:
# One merge iteration
new_ix += 1
# Find most frequent pair. Note: the pair popped from the heap can be invalid if it was updated
# as an overlapping pair in the actual pair count dictionary. That's why we need to compare it
# with the actual state.
while True:
  neg_cnt, neg_p = heapq.heappop(heap)
  # if count is correct we are fine to break
  if -neg_cnt == pairs_count[(-neg_p[0], -neg_p[1])]: 
    break
  else: # invalid entry
    continue
top_pair = (-neg_p[0], -neg_p[1])
print(top_pair, vocab[top_pair[0]], vocab[top_pair[1]])

(119, 257) b'w' b'est'


In [92]:
heap

[(-9, (-101, -115)),
 (-8, (-119, -101)),
 (-7, (-111, -119)),
 (-3, (-119, -105)),
 (-7, (-108, -111)),
 (-6, (-110, -101)),
 (-6, (-101, -119)),
 (-3, (-105, -100)),
 (-3, (-100, -101)),
 (-7, (-32, -108)),
 (-2, (-101, -114)),
 (-6, (-32, -110)),
 (-3, (-32, -119))]

In [93]:
# now we can walk through only pretoken sequences actually containing the top pair
in_pairs = set()
for pt in pair_to_pretoken[top_pair]:
  pt_cnt = pretoken_count[pt] # current pretoken frequency
  # during merging we also want to collect the set of outgoing and incoming pairs (overlapping)
  # we have to keep incoming pairs to update heap after we have all pair counts updated
  # while outgoing pairs being lazily updated during our "while true heappop"
  new_pt, out_pairs, in_pairs = merge(pt, top_pair, new_ix, in_pairs)
  # update all our datastructures
  for op in out_pairs:
    # overall frequency of op is now pt_cnt less
    pairs_count[op] -= pt_cnt 
    # discard old pretoken from the set of pretokens containing outgoing pair
    pair_to_pretoken[op].discard(pt) 
  for ip in in_pairs:
    # frequency of incoming pair has increased
    pairs_count[ip] += pt_cnt
    # add new pretoken to the set of pretokens containing incoming pair
    pair_to_pretoken[ip].add(new_pt)
  # pretoken counts need to be updated as well
  pretoken_count[pt] -= pt_cnt
  pretoken_count[new_pt] = pretoken_count.get(new_pt, 0) + pt_cnt
# we need to update heap with incoming pairs
for ip in in_pairs:
  heapq.heappush(heap, (-pairs_count[ip], (-ip[0], -ip[1])))
# finished working with current top pair, so we need to zero its state
pairs_count[top_pair] = 0
pair_to_pretoken[top_pair].clear()



In [94]:
pretoken_count

{(32, 108, 111, 119): 5,
 (32, 108, 111, 119, 101, 114): 2,
 (32, 119, 105, 100, 101, 115, 116): 0,
 (32, 110, 101, 119, 101, 115, 116): 0,
 (32, 110, 101, 119, 101, 256): 0,
 (32, 119, 105, 100, 101, 256): 0,
 (32, 110, 101, 119, 257): 6,
 (32, 119, 105, 100, 257): 3}

In [95]:
pairs_count

defaultdict(int,
            {(32, 108): 7,
             (108, 111): 7,
             (111, 119): 7,
             (119, 101): 2,
             (101, 114): 2,
             (32, 119): 3,
             (119, 105): 3,
             (105, 100): 3,
             (100, 101): 0,
             (101, 115): 0,
             (115, 116): 0,
             (32, 110): 6,
             (110, 101): 6,
             (101, 119): 6,
             (101, 256): 0,
             (119, 257): 9,
             (100, 257): 3})

In [96]:
pair_to_pretoken

defaultdict(set,
            {(32, 108): {(32, 108, 111, 119), (32, 108, 111, 119, 101, 114)},
             (108, 111): {(32, 108, 111, 119), (32, 108, 111, 119, 101, 114)},
             (111, 119): {(32, 108, 111, 119), (32, 108, 111, 119, 101, 114)},
             (119, 101): {(32, 108, 111, 119, 101, 114),
              (32, 110, 101, 119, 101, 115, 116)},
             (101, 114): {(32, 108, 111, 119, 101, 114)},
             (32, 119): {(32, 119, 105, 100, 101, 115, 116)},
             (119, 105): {(32, 119, 105, 100, 101, 115, 116)},
             (105, 100): {(32, 119, 105, 100, 101, 115, 116)},
             (100, 101): {(32, 119, 105, 100, 101, 115, 116)},
             (101, 115): set(),
             (115, 116): set(),
             (32, 110): {(32, 110, 101, 119, 101, 115, 116)},
             (110, 101): {(32, 110, 101, 119, 101, 115, 116)},
             (101, 119): {(32, 110, 101, 119, 101, 115, 116)},
             (101, 256): set(),
             (119, 257): {(32, 110, 101, 119, 

In [97]:
heap

[(-9, (-119, -257)),
 (-8, (-119, -101)),
 (-9, (-101, -115)),
 (-3, (-119, -105)),
 (-7, (-108, -111)),
 (-6, (-110, -101)),
 (-7, (-111, -119)),
 (-3, (-105, -100)),
 (-3, (-100, -101)),
 (-7, (-32, -108)),
 (-2, (-101, -114)),
 (-6, (-32, -110)),
 (-3, (-32, -119)),
 (-6, (-101, -119)),
 (-3, (-100, -257))]

In [98]:
# update merges and vocab
merges.append(top_pair)
vocab[new_ix] = vocab[top_pair[0]] + vocab[top_pair[1]]

In [99]:
merges

[(115, 116), (101, 256)]

In [100]:
vocab[new_ix]

b'est'